Cleaner version of the notebook that I run on laptop to scape data faster

In [1]:
#import libraries
from time import time, sleep
import random
import urllib
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests
import bs4
from bs4 import BeautifulSoup
import re
from IPython.display import clear_output
import os

In [2]:
#captcha alert settings
import winsound
duration = 5000  # milliseconds
freq = 440  # Hz

In [3]:
headers_list = [
# Firefox 77 Mac
{
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Referer": "https://www.google.com/",
"DNT": "1",
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1"
},
# Firefox 77 Windows
{
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
"Accept-Language": "en-US,en;q=0.5",
"Accept-Encoding": "gzip, deflate, br",
"Referer": "https://www.google.com/",
"DNT": "1",
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1"
},
# Chrome 83 Mac
{
"Connection": "keep-alive",
"DNT": "1",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "none",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-Dest": "document",
"Referer": "https://www.google.com/",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
},
# Chrome 83 Windows 
{
"Connection": "keep-alive",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "same-origin",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-User": "?1",
"Sec-Fetch-Dest": "document",
"Referer": "https://www.google.com/",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-US,en;q=0.9"
}
]

In [4]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 5000
cities = ['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
                 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
                 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Boston',
                 'Charlottesville', 'Richmond', 'Baltimore', 'Harrisonburg', 'San+Antonio', 'San+Diego', 'San+Jose',
                 'Austin', 'Jacksonville', 'Indianapolis', 'Columbus', 'Fort+Worth', 'Charlotte', 'Detroit', 'El+Paso', 
                 'Memphis', 'Orlando', 'Nashville', 'Louisville', 'Milwaukee', 'Las+Vegas', 'Albuquerque', 'Tucson', 
                 'Fresno', 'Sacramento', 'Long+Beach', 'Mesa', 'Virginia+Beach', 'Norfolk', 'Atlanta', 'Colorado+Springs',
                 'Raleigh', 'Omaha', 'Oakland', 'Tulsa', 'Minneapolis', 'Cleveland', 'Wichita', 'Arlington', 'New+Orleans', 
                 'Bakersfield', 'Tampa', 'Honolulu', 'Anaheim', 'Aurora', 'Santa+Ana', 'Riverside', 'Corpus+Christi', 
                 'Pittsburgh', 'Lexington', 'Anchorage', 'Cincinnati', 'Baton+Rouge', 'Chesapeake', 'Alexandria', 'Fairfax', 
                 'Herndon','Reston', 'Roanoke', 'Boulder', 'Pasadena', 'Greensboro', 'Newark']

In [5]:
i = 0
count = 0
results = []

for city in set(cities):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        file = 'data_by_cities/'+str(city)+'.csv'
        try: 
            df_more = pd.read_csv(file , sep='\t', encoding='utf-8', index_col=0)
            print('file successfully opened')
        except:
            print('file not found, creating new file')
            df_more = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis"])
            df_more.to_csv(file, sep='\t', encoding='utf-8')
        datasize = df_more.shape
        url = url_template.format(city, start)
        headers = random.choice(headers_list)
        r = requests.Session()
        r.headers = headers
        html = r.get(url)
        #html = requests.get(url)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        #captcha detector
        if 'hCaptcha solve page' in soup.get_text():
            print('captcha detected')
            winsound.Beep(freq, duration) #sound alert, might get irritating, probably won't work on mac
            input("Press Enter to continue...")#this would pause the code until i press enter
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobTitle').text.replace('\n', '').replace('new', '')
            except:
                title = 'None'
            try:
                location = each.find('div', {'class':"companyLocation" }).text.replace('\n', '')
            except:
                location = 'None'
            try: 
                company = each.find(class_='companyName').text.replace('\n', '')
            except:
                company = 'None'
            try:
                salary = each.find('div', {'class': "metadata salary-snippet-container"}).text.replace('\n', '')
            except:
                try:
                    salary = each.find('span', {'class':'estimated-salary'}).text.replace('\n', '')
                except:
                    salary = 'None'
            synopsis = each.find('div', {'class':'job-snippet'}).text.replace('\n', '')
            df_more = df_more.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis}, ignore_index=True)
            #drop duplicates
            df_more = df_more[~df_more.duplicated()].reset_index(drop=True)
        #count the number of pages scraped and print, so I know how far the code is running
        count+= 1
        print(count, ' loops done, proceed to sleep, current city is ', city)
        #move to next city if we don't get new data. it's pointless to continue without getting new data.
        if df_more.shape == datasize:
            print(df_more.shape,' no new data, moving to next city')
            break       
        df_more.to_csv(file, sep='\t', encoding='utf-8')#write every page so i don't lose my data again
        sleep(random.randint(7,20))
        #I don't want too see long line of the process, clear_output will clear previous lines
        clear_output(wait=True)
        print(df_more.shape, ' data saved, last visited city is ', city, ', continuing....')
    print('moving to next city')
    sleep(random.randint(10,30))

(189, 5)  data saved, last visited city is  Dallas , continuing....
file successfully opened
544  loops done, proceed to sleep, current city is  Dallas
(189, 5)  no new data, moving to next city
moving to next city


In [ ]:
df_more